# Flat Image Net - Basic Graph


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import shutil
from utils.data import init_dir

## Load data

In [4]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

Extracting /data/fashion/train-images-idx3-ubyte.gz
Extracting /data/fashion/train-labels-idx1-ubyte.gz
Extracting /data/fashion/t10k-images-idx3-ubyte.gz
Extracting /data/fashion/t10k-labels-idx1-ubyte.gz


## Basic summary functions

In [5]:
def variable_summaries(name, var):
    with tf.name_scope(name):        
        tf.summary.histogram('histogram', var)
        ###########################################################
        # Complete scalar summaries for min, max, stddev and mean #
        ###########################################################
        
def img_summaries(name, var):
    with tf.name_scope(name):
        tf.summary.image(name, var)

## Custom layer functions

In [6]:
def simple_layer(name, input_data, shape, activation='linear'):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    w = tf.get_variable('w', initializer=tf.truncated_normal(shape, stddev=0.1))
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0.1), shape=shape[1])
    
    #####################################################
    # Complete xavier and variance_scaling initializers #
    #####################################################
    
    output_data = tf.matmul(input_data, w) + bias
    
    ######################################################
    # Complete sigmoid,tanh, linear and relu activations #
    ######################################################
    
    return output_data

## Build Net Graph

In [9]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('flat_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')
    
    with tf.variable_scope('simple_layer_1'):
        raw_prediction = simple_layer(name='layer1', input_data=images, shape=[784, 10])
    #####################
    # Add another layer #
    #####################
    
    ###############
    # Use dropout #
    ###############
            
    with tf.name_scope('prediction'):
        prediction = raw_prediction
        #####################
        # Fix prediction    #
        #####################
    
    with tf.name_scope('loss'):
        #####################
        # Fix loss function #
        #####################
        loss_vector = raw_prediction - labels
        loss = tf.reduce_mean(loss_vector)
        print('Debug [loss_vector]:', loss_vector.get_shape())
        variable_summaries('loss_summary', loss_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)       
        
    with tf.name_scope('training'):
        ####################
        # Change optimizer #
        ####################
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
            
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

Debug [loss_vector]: (?, 10)


## Init Model Logging

In [ ]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/flat_image_net/'

logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

## Debug Net

In [11]:
with tf.Session(graph=graph) as session:
    session.run(initialize_vars)
    _images, _labels = data.train.next_batch(10)
    feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 1}
    print("Debug [loss_vector]:", session.run(tf.shape(loss_vector), feed_dict))

Debug [loss_vector]: [10 10]


## Run Net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']
        
with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    for iteration in range(10000):
        ##################
        # Training phase #
        ##################
        _images, _labels = data.train.next_batch(100)
        _ = session.run([train_step], feed_dict={images: _images, labels: _labels, keep_dropout_prob: 0.5})
        if iteration % 10 == 0:
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss],
                                                     feed_dict={images: _images, 
                                                                labels: _labels, 
                                                                keep_dropout_prob: 1.0})
            logging_meta['train_writer'].add_summary(_summary, iteration)
            print("Train Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
      
        ####################
        # Validation phase #
        ####################
        if iteration % 100 == 0:
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss], 
                                          feed_dict={images: data.validation.images, 
                                                     labels: data.validation.labels,
                                                     keep_dropout_prob: 1.0})
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))
            
    _prediction, = session.run([prediction], feed_dict={images: data.validation.images, keep_dropout_prob: 1.0})

In [ ]:
_prediction